In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import missingno as msno
import statsmodels.api as sm
#import tensorflow as tf
#imp
df = pd.read_csv(r"C:\Users\Hamza\Desktop\P2P ortak dosya\Makine Öğrenmesi\Solar1\solar_predict.xls")   #dataframe determined
df.columns=["Date","Time","Power","Load","Temp"]   #the columns in our dataframe
from datetime import datetime
from pytz import timezone
import pytz
df.index=df.iloc[:,0]
df.index = pd.to_datetime(df.index)  #convert to date
df.drop(['Load'], axis=1, inplace=True)   #I will just look at PV Generation
df['MonthOfYear'] = df.index.strftime('%m').astype(int)
df['DayOfYear'] = df.index.strftime('%j').astype(int)
df['WeekOfYear'] = df.index.strftime('%U').astype(int)
df.index=df.iloc[:,1]
df.index = pd.to_datetime(df.index)
df['TimeOfDay'] = df.index.hour
from pytz import timezone
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import neighbors
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Range1d
#from tensorflow.keras.layers import Dense,Activation,Dropout
#from tensorflow.keras.models import Sequential,load_model
#from tensorflow.keras.optimizers import SGD, Adam
#from tensorflow.keras.models import Sequential
from collections import Counter
from scipy import stats
from sklearn.linear_model import LinearRegression, BayesianRidge, Ridge ,Lasso ,ElasticNet, LassoLarsIC
from sklearn.linear_model import RidgeCV, LassoCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import KFold,RandomizedSearchCV
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, scale, LabelEncoder  
from sklearn.impute import SimpleImputer
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor 
from sklearn.svm import SVR
#from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
#from xgboost import XGBRegressor
from warnings import filterwarnings
filterwarnings('ignore')
df = pd.read_csv(r"C:\Users\Hamza\Desktop\P2P ortak dosya\Makine Öğrenmesi\Solar1\solar_predict.xls")   #dataframe determined
df.columns=["Date","Time","Power","Load","Temp"]   #the columns in our dataframe
from datetime import datetime
from pytz import timezone
import pytz
df.index=df.iloc[:,0]
df.index = pd.to_datetime(df.index)  #convert to date
df.drop(['Load'], axis=1, inplace=True)   #I will just look at PV Generation
df['MonthOfYear'] = df.index.strftime('%m').astype(int)
df['DayOfYear'] = df.index.strftime('%j').astype(int)
df['WeekOfYear'] = df.index.strftime('%U').astype(int)
df.index=df.iloc[:,1]
df.index = pd.to_datetime(df.index)
df['TimeOfDay'] = df.index.hour

ModuleNotFoundError: No module named 'lightgbm'

In [7]:
pip install lightbgm

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement lightbgm (from versions: none)
ERROR: No matching distribution found for lightbgm


In [3]:
X = df[['MonthOfYear', 'DayOfYear', 'WeekOfYear',"TimeOfDay","Temp"]]
y = df.iloc[:,2:3]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
from lightgbm import LGBMRegressor
lgb_model = LGBMRegressor().fit(X_train, y_train)
y_pred = lgb_model.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

ModuleNotFoundError: No module named 'lightgbm'

In [ ]:
lgb_model = LGBMRegressor()
lgbm_params = {"learning_rate": [0.01, 0.1, 0.5, 1],
              "n_estimators": [20,40,100,200,500,1000],
              "max_depth": [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]}

In [ ]:
lgbm_cv_model = GridSearchCV(lgb_model, 
                             lgbm_params, 
                             cv = 10,  
                             verbose =2).fit(X_train, y_train)

In [ ]:

lgbm_cv_model.best_params_

lgbm_tuned = LGBMRegressor(learning_rate = 0.1, 
                          max_depth = 15, 
                          n_estimators = 1000).fit(X_train, y_train)

In [ ]:

ypred=lgbm_tuned.predict(X_test)
print("mse = ",np.sqrt(mean_squared_error(y_test,ypred)))
model=r2_score(y_test, y_pred)
print('r2 = {}'.format(model)) 

In [ ]:
df.index=df.iloc[:,0]
df.index = pd.to_datetime(df.index)
# Predicting for whole dataset
df['y_pred'] =  lgbm_tuned.predict(df[['MonthOfYear', 'DayOfYear', 'WeekOfYear',"TimeOfDay","Temp"]])

In [ ]:
# Create Interactive Plot Using Bokeh
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Range1d
output_notebook() 


In [ ]:
p = figure(plot_width=800, 
           plot_height=400,
           title="FV Elektrik Üretim Tahmini",
           x_axis_label='Tarih ve Zaman',
           y_axis_label='Elektrik Üretimi[kW]',
           x_axis_type="datetime")


In [ ]:
p.line(df.index, df['Power'], legend='Gözlemlenen Elektrik Üretimi', line_width=1)
p.line(df.index, df['y_pred'], legend='Tahmin edilen Elektrik Üretimi ', line_width=1, line_color="red")

In [ ]:
show(p, notebook_handle = True)

In [ ]:
Importance = pd.DataFrame({'Importance':lgbm_tuned.feature_importances_*100}, 
                          index = X_train.columns)


In [ ]:
Importance.sort_values(by = 'Importance', 
                       axis = 0, 
                       ascending = True).plot(kind = 'barh', 
                                              color = 'r', )

In [ ]:
plt.xlabel('Variable Importance')
plt.gca().legend_ = None

In [ ]:
day=[x for x in range(0,24)]
temp1=[15,15,14,14,14,14,15,16,17,17,18,18,18,18,19,18,17,16,15,14,13,13,12,12]
print("---------------12-1------------")
for i in range(0,24):
    print((regressor.predict([[12,18,3,day[i],temp1[i]]])))

temp2=[21,21,20,20,20,19,19,20,21,23,24,26,27,29,29,25,23,22,20,20,19,18,18,17]
print("---------------1-4------------")
for i in range(0,24):
    print((regressor.predict([[12,18,3,day[i],temp2[i]]])))

temp3=[8,7,6,6,5,5,5,4,5,8,10,11,12,13,13,12,11,10,10,9,9,8,8,7]
print("---------------1-7------------")
for i in range(0,24):
    print((regressor.predict([[12,18,3,day[i],temp3[i]]])))

temp4=[12,12,11,10,10,10,9,11,14,17,20,21,22,23,23,23,22,21,18,16,15,14,13,13]
print("---------------1-10------------")
for i in range(0,24):
    print((regressor.predict([[12,18,3,day[i],temp4[i]]])))